# Para instalar una biblioteca faltante: pip install "nombre biblioteca"

In [24]:
import numpy as np
import pandas as pd
import re as regex
import math as math
import requests as req #biblioteca para hacer peticiones HTTP a servicios: pip install request

In [2]:
pathArchivoDataSet = 'properatti.csv'
ds = pd.read_csv(pathArchivoDataSet)

In [3]:
ds.isnull().sum(axis=0)

Unnamed: 0                         0
operation                          0
property_type                      0
place_name                        23
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    18717
lat-lon                        51550
lat                            51550
lon                            51550
price                          20410
currency                       20411
price_aprox_local_currency     20410
price_aprox_usd                20410
surface_total_in_m2            39328
surface_covered_in_m2          19907
price_usd_per_m2               52603
price_per_m2                   33562
floor                         113321
rooms                          73830
expenses                      106958
properati_url                      0
description                        2
title                              0
image_thumbnail                 3112
dtype: int64

# Obtener datos de lat y long a partir del geonames_id, esto no da la ubicación exacta de la propiedad, sino uno coordenada dentro de una zona (por ej. Carlos Paz o San Fernando)

### La siguiente es la URL del servicio, requiere un el nombre de un usuario generado en el sitio https://www.geonames.org/ y un geonameId, obtenido de la columna geonames_id (que vamos a ir obteniendo mientras vamos leyendo el data frame

In [19]:
urlServicio = "http://api.geonames.org/getJSON"

### Parametros de la request

In [20]:
PARAMS = { 'username': 'rdgagliano', 'style': 'full', 'geonameId': 3429088 }

### Realizar la request

In [21]:
r = req.get(url = urlServicio, params = PARAMS)

In [22]:
r.json()

{'timezone': {'gmtOffset': -3,
  'timeZoneId': 'America/Argentina/Buenos_Aires',
  'dstOffset': -3},
 'bbox': {'east': -58.35540771500001,
  'south': -34.489437103,
  'north': -34.00189209000001,
  'west': -58.805011749000016,
  'accuracyLevel': 0},
 'asciiName': 'Partido de San Fernando',
 'astergdem': 20,
 'countryId': '3865483',
 'fcl': 'A',
 'srtm3': 21,
 'adminId2': '3429088',
 'countryCode': 'AR',
 'adminCodes1': {'ISO3166_2': 'B'},
 'adminId1': '3435907',
 'lat': '-34.44542',
 'fcode': 'ADM2',
 'continentCode': 'SA',
 'adminCode2': '06749',
 'adminCode1': '01',
 'lng': '-58.54202',
 'geonameId': 3429088,
 'toponymName': 'Partido de San Fernando',
 'population': 163462,
 'wikipediaURL': 'en.wikipedia.org/wiki/San_Fernando_Partido',
 'adminName5': '',
 'adminName4': '',
 'adminName3': '',
 'alternateNames': [{'name': 'https://en.wikipedia.org/wiki/San_Fernando_Partido',
   'lang': 'link'},
  {'name': 'Partido di San Fernando', 'lang': 'it'},
  {'name': 'Partido San Fernando', 'lan

### Con la respuesta podemos sacar la lat y long para completar aquellas filas del data frame que tengan el geonames_id pero no lat, long ni lat-long

# Verificamos cuantos campos lat y lon pueden llegar a completarse a partir del geonames_id

In [33]:
# Cantidad de registros que tienen geonames_id pero alguno de lat, lon o lat-lon está vacío
len(ds.loc[~ds['geonames_id'].isnull() & (ds['lat'].isnull() | ds['lon'].isnull() | ds['lat-lon'].isnull())])

43365

### Eliminamos columnas innecesarias para el procesamiento

In [34]:
ds.drop(ds.columns.difference(['geonames_id','lat', 'lon', 'lat-lon']), 1, inplace=True)

In [35]:
ds.head(5)

,geonames_id,lat-lon,lat,lon
0,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839
1,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330
2,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982
3,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424
4,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447


### Solo nos quedamos con las filas en las que tengan el geonames_id, pero que no tengan lat o lon

In [105]:
ds_slice = ds.loc[~ds['geonames_id'].isnull() & (ds['lat'].isnull() | ds['lon'].isnull() | ds['lat-lon'].isnull())]

In [112]:
ds_slice.reset_index()

,index,geonames_id,lat-lon,lat,lon
0,13,3430234.0,NaN,NaN,NaN
1,42,3429979.0,NaN,NaN,NaN
2,49,3434130.0,NaN,NaN,NaN
3,62,3436042.0,NaN,NaN,NaN
4,86,3862356.0,NaN,NaN,NaN
...,...,...,...,...,...
458,116062,3832705.0,NaN,NaN,NaN
459,116479,3861887.0,NaN,NaN,NaN
460,117229,3429610.0,NaN,NaN,NaN
461,118851,3431318.0,NaN,NaN,NaN


### Quitamos repetidos para no hacer peticiones innecesarias

In [117]:
ds_slice = ds_slice.drop_duplicates(subset = ['geonames_id'], keep='first')

In [135]:
def setLatLonFromGeonameId(dataFrameRow):    
    geonamesId = int(dataFrameRow.geonames_id)        
    
    #preparamos la peticion
    urlJson = "http://api.geonames.org/getJSON"
    parametros = { 'username': 'rdgagliano', 'style': 'full', 'geonameId': geonamesId }
    
    #realizamos la peticion
    respuesta = req.get(url = urlJson, params = parametros)
    
    #el objeto json es un diccionario
    json = respuesta.json()    
    
    try:
        dataFrameRow['lat'] = float(json['lat'])#se accede a las propiedades de json como diccionario
        dataFrameRow['lon'] = float(json['lng'])
        dataFrameRow['lat-lon'] = json['lat'] + ',' + json['lng']
    except:
        print(json)
        print(geonamesId)
    
    return dataFrameRow

In [136]:
ds_slice = ds_slice.apply(setLatLonFromGeonameId, axis=1)

{'status': {'message': 'the geoname feature does not exist.', 'value': 11}}
3435548


In [138]:
ds_slice[ds_slice.lat.isnull()]

,geonames_id,lat-lon,lat,lon
6109,3435548.0,NaN,NaN,NaN


### Nos quedamos solamente con los que se pudieron completar

In [142]:
ds_slice = ds_slice.loc[~ds['geonames_id'].isnull() & (~ds_slice['lat'].isnull() & ~ds_slice['lon'].isnull() & ~ds_slice['lat-lon'].isnull())]

# Para ahorrar el tiempo que tarda en hacer todas las requests, guardamos los datos obtenidos en un CSV para su posterior carga

### Generamos el CSV del dataframe resultante

In [149]:
latLngCSVFileName = 'latLngFromGeonames.csv'

In [150]:
ds_slice.to_csv(latLngCSVFileName,index=False,encoding='utf-8')

### Cargamos el archivo

In [151]:
latLong = pd.read_csv(latLngCSVFileName)

In [152]:
latLong.head()

,geonames_id,lat-lon,lat,lon
0,3430234.0,"-34.58856,-58.43053",-34.58856,-58.43053
1,3429979.0,"-34.41667,-58.83333",-34.41667,-58.83333
2,3434130.0,"-34.33333,-58.75",-34.33333,-58.75000
3,3436042.0,"-34.83333,-58.16667",-34.83333,-58.16667
4,3862356.0,"-36.59806,-61.74306",-36.59806,-61.74306


# Completamos los datos de latitud y longitud a partir del CSV

In [186]:
ds2 = pd.read_csv(pathArchivoDataSet)

In [191]:
ds2.head()

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...


In [199]:
ds2['lat'].isna().sum() #contamos los vacíos en lat

51550

In [198]:
def updateCoordinates(dataFrameRow):
    
    if(not math.isnan(dataFrameRow['geonames_id'])):
        geonameIdValue = int(dataFrameRow['geonames_id'])

        #print(geonameIdValue)
        
        coordinatesData = latLong.loc[latLong['geonames_id'] == geonameIdValue]
        
        if(coordinatesData is not None and len(coordinatesData) == 1):
            #print(coordinatesData.lat)
            #print(coordinatesData.lon)
            #print(coordinatesData['lat-lon'])

            dataFrameRow.lat = coordinatesData.lat
            dataFrameRow.lon = coordinatesData.lon
            dataFrameRow['lat-lon'] = coordinatesData['lat-lon']
    
    return dataFrameRow

In [200]:
ds2 = ds2.apply(updateCoordinates, axis=1)

In [201]:
ds2['lat'].isna().sum() #contamos los vacíos en lat luego del cambio

8346